# BMP Header
Reads an bmp header and displays it's values. See also: http://wpwiki/doku.php?id=internal:general:bmp_file_specification

**Copy from HxD Editor beginning of bmp file**

Works with 
* Gimp BMP
* Ergosoft Rip BMP (Number of colors are not written)
* ImageEditor BMP

**Example usage see below**

In [10]:
def bmp_header_analysis(bmp_header = "42 4D 76 30 D9 00 00 00 00 00 76 00 00 00 28 00 00 00 00 19 00 00 60 11 00 00 01 00 04 00 00 00 00 00 00 30 D9 00 46 5C 00 00 46 5C 00 00 00 00 00 00 00 00 00 00 FF FF FF 00 80 80 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00"):
  bmp_header_def = [[0, 2, "BMP Header Field", ""],
                    [2, 4, "BMP file size", ""],
                    [6, 2, "Unused 1", ""],
                    [8, 2, "Unused 2", ""],
                    [10, 4, "Offset of BMP Data", ""],
                    [14, 4, "Number of Bytes in DIP Header", ""],
                    [18, 4, "Width of Bitmap in Pixels", ""],
                    [22, 4, "Height of Bitmap in Pixels", ""],
                    [26, 2, "Number of color planes being used", ""],
                    [28, 2, "bits/pixels", ""],
                    [30, 4, "Pixel Array Compression", ""],
                    [34, 4, "Size of Raw Bitmap Data", ""],
                    [38, 4, "pixels/meter X Print Resolution", ""],
                    [42, 4, "pixels/meter Y Print Resolution", ""],
                    [46, 4, "Numbers of color in palette", ""],
                    [50, 4, "Important Colors", ""],
                    [1000, 3, "Color 1", ""],           # offset address will be calculated
                    [1000, 3, "Color 2", ""],           # offset address will be calculated
                    [1000, 3, "Color 3", ""],           # offset address will be calculated
                    [1000, 3, "Color 4", ""],           # offset address will be calculated
                    [1000, 2000, "Raw Bitmap Data", ""] # offset address will be calculated
  ]
  bmp_header_size = 14

  # Clean bmp header
  bmp_header = bmp_header.replace(" ", "")

  # Split bmp header in byte pieces
  i = 0
  bmp_header_bytes = []
  while i < len(bmp_header):
    #print("{0} {1}".format(i, bmp_header[i]))
    bmp_header_bytes.append(bmp_header[i] + bmp_header[i+1])
    i += 2
  
  #print bmp_header_bytes
  
  # Get Header Fields according to spec
  for i in range(0, len(bmp_header_bytes)):
    for j in range(0, len(bmp_header_def)):
      if i >= bmp_header_def[j][0] and i < (bmp_header_def[j][0] + bmp_header_def[j][1]):
        #print("Field {0} found".format(bmp_header_analysis[j][2]))
        bmp_header_def[j][3] = bmp_header_bytes[i] + bmp_header_def[j][3]
        # write correct offset of color table and raw bitmap data
        if j == 5:
          value   = int(bmp_header_def[j][3],16)
          bmp_header_def[16][0] = bmp_header_size + value
          bmp_header_def[17][0] = bmp_header_def[16][0] + bmp_header_def[16][1] + 1
          bmp_header_def[18][0] = bmp_header_def[17][0] + bmp_header_def[17][1] + 1 
          bmp_header_def[19][0] = bmp_header_def[18][0] + bmp_header_def[18][1] + 1
          bmp_header_def[20][0] = bmp_header_def[19][0] + bmp_header_def[19][1] + 1
  
  # Analysis
  print("Field name                        | Hex Value  | Add Value  | Comment")
  print("----------------------------------+------------+------------+----------------------------------------")
  error = False
  for field in bmp_header_def:
    comment = "Unknown"
    value = ""
    if field[2] == "BMP Header Field" and field[3] == "4D42":
      if field[3] == "4D42":
        comment = "BMP Header Field detected => CORRECT"
        value   = "BM"
      else:
        comment = "BMP Header Field detected => ERROR"
        value   = "-"
        error = True
    elif field[2] == "BMP file size":
      comment = "Filesize in bytes"
      value   = int(field[3], 16)
      if value > 1024:
        value   = round(float(value)/1024,4)
        comment = "kbytes Filesize"
      if value > 1024:
        value   = round(float(value)/1024,4)
        comment = "Mbytes Filesize"
      if value > 1024:
        value   = round(float(value)/1024,4)
        comment = "Gbytes Filesize"
    elif field[2] == "Unused 1" or field[2] =="Unused 2":
      value   = "-"
      comment = "Unused Application specific settings"
    elif field[2] == "Offset of BMP Data":
      value   = int(field[3], 16)
      comment = field[2]
    elif field[2] == "Number of Bytes in DIP Header":
      value   = int(field[3],16)
      comment = field[2]
    elif field[2] == "Width of Bitmap in Pixels":
      value   = int(field[3],16)
      comment = "px Width of Bitmap"
    elif field[2] == "Height of Bitmap in Pixels":
      value   = int(field[3],16)
      comment = "px Height of Bitmap"
    elif field[2] == "Number of color planes being used":
      value   = int(field[3],16)
      comment = field[2]
    elif field[2] == "bits/pixels":
      value   = int(field[3],16)
      if value == 4:
        comment = "bits/pixels => CORRECT"
      else:
        comment = "bits/pixels => ERROR"
        error = True
    elif field[2] == "Pixel Array Compression":
      value   = int(field[3],16)
      comment = field[2]
    elif field[2] == "Size of Raw Bitmap Data":
      value   = int(field[3], 16)
      comment = "Raw Bitmap data in bytes"
      if value > 1024:
        value   = round(float(value)/1024,4)
        comment = "kbytes Raw Bitmap data"
      if value > 1024:
        value   = round(float(value)/1024,4)
        comment = "Mbytes Raw Bitmap data"
      if value > 1024:
        value   = round(float(value)/1024,4)
        comment = "Gbytes Raw Bitmap data"
    elif field[2] == "pixels/meter X Print Resolution":
      value   = float(int(field[3],16))/(100.0)*2.54
      comment = "dpi X Print Resolution"
    elif field[2] == "pixels/meter Y Print Resolution":
      value   = float(int(field[3],16))/(100.0)*2.54
      comment = "dpi Y Print Resolution"
    elif field[2] == "Numbers of color in palette":
      value   = int(field[3],16)
      comment = field[2]
      if value == 4:
        comment = "Colors in Palette => CORRECT"
      else:
        comment = "Colors in Palette => ERROR"
        error = True
    elif field[2] == "Important Colors":
      value   = int(field[3],16)
      comment = field[2]
    elif field[2] == "Color 1":
      value   = field[3]
      comment = "display color for pixel value 0"
    elif field[2] == "Color 2":
      value   = field[3]
      comment = "display color for pixel value 1"
    elif field[2] == "Color 3":
      value   = field[3]
      comment = "display color for pixel value 2"
    elif field[2] == "Color 4":
      value   = field[3]
      comment = "display color for pixel value 3"
    elif field[2] == "Raw Bitmap Data":
      value   = "-"
      comment = field[2]
     
    if len(field[3]) <= 8:
      print("{0:33} | {1:10} | {2:10} | {3}".format(field[2], field[3], value, comment))
    else:
      print("{0:33} | see below  | {1:10} | {2}".format(field[2], value, comment))
      print(field[3])
      
  
  print("-----------------------------------------------------------------------------------------------------")
  if error:
    print("Header has an Error: see Table above")
  else:
    print("Header is CORRECT")
  print("-----------------------------------------------------------------------------------------------------")

### ErgoSoft Header Example

In [4]:
bmp_header = "42 4D 76 30 D9 00 00 00 00 00 76 00 00 00 28 00 00 00 00 19 00 00 60 11 00 00 01 00 04 00 00 00 00 00 00 30 D9 00 46 5C 00 00 46 5C 00 00 00 00 00 00 00 00 00 00 FF FF FF 00 80 80 80 00 00 00 00 00 00 00 00 00 00 00 00 00"

bmp_header_analysis(bmp_header)

Field name                        | Hex Value  | Add Value  | Comment
----------------------------------+------------+------------+----------------------------------------
BMP Header Field                  | 4D42       | BM         | BMP Header Field detected => CORRECT
BMP file size                     | 00D93076   |    13.5743 | Mbytes Filesize
Unused 1                          | 0000       | -          | Unused Application specific settings
Unused 2                          | 0000       | -          | Unused Application specific settings
Offset of BMP Data                | 00000076   |        118 | Offset of BMP Data
Number of Bytes in DIP Header     | 00000028   |         40 | Number of Bytes in DIP Header
Width of Bitmap in Pixels         | 00001900   |       6400 | px Width of Bitmap
Height of Bitmap in Pixels        | 00001160   |       4448 | px Height of Bitmap
Number of color planes being used | 0001       |          1 | Number of color planes being used
bits/pixels          

### Gimp Header Example

In [6]:
"42 4D 8A 00 DC 00 00 00 00 00 8A 00 00 00 6C 00 00 00 00 14 00 00 00 16 00 00 01 00 04 00 00 00 00 00 00 00 DC 00 13 0B 00 00 13 0B 00 00 04 00 00 00 04 00 00 00 42 47 52 73 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 02 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 FD FD FD 00 99 99 99 00 66 66 66 00 03 03 03 00 33 33 33 33"

bmp_header_analysis(bmp_header)

Field name                        | Hex Value  | Add Value  | Comment
----------------------------------+------------+------------+----------------------------------------
BMP Header Field                  | 4D42       | BM         | BMP Header Field detected => CORRECT
BMP file size                     | 00271436   |     2.4424 | Mbytes Filesize
Unused 1                          | 0000       | -          | Unused Application specific settings
Unused 2                          | 0000       | -          | Unused Application specific settings
Offset of BMP Data                | 00000436   |       1078 | Offset of BMP Data
Number of Bytes in DIP Header     | 00000028   |         40 | Number of Bytes in DIP Header
Width of Bitmap in Pixels         | 000007D0   |       2000 | px Width of Bitmap
Height of Bitmap in Pixels        | 00000500   |       1280 | px Height of Bitmap
Number of color planes being used | 0001       |          1 | Number of color planes being used
bits/pixels          

## Test your own Header here ;-)

In [7]:
bmp_header ="42 4D C6 89 08 00 00 00 00 00 46 00 00 00 28 00 00 00 67 04 00 00 E0 03 00 00 01 00 04 00 00 00 00 00 00 00 00 00 C4 0E 00 00 C4 0E 00 00 04 00 00 00 04 00 00 00 FF FF FF FF C0 C0 C0 FF 81 81 81 FF 42 42 42 FF 00 00 00 00 00 00 00 00 00 00 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33 33"
bmp_header_analysis(bmp_header)

Field name                        | Hex Value  | Add Value  | Comment
----------------------------------+------------+------------+----------------------------------------
BMP Header Field                  | 4D42       | BM         | BMP Header Field detected => CORRECT
BMP file size                     | 000889C6   |   546.4434 | kbytes Filesize
Unused 1                          | 0000       | -          | Unused Application specific settings
Unused 2                          | 0000       | -          | Unused Application specific settings
Offset of BMP Data                | 00000046   |         70 | Offset of BMP Data
Number of Bytes in DIP Header     | 00000028   |         40 | Number of Bytes in DIP Header
Width of Bitmap in Pixels         | 00000467   |       1127 | px Width of Bitmap
Height of Bitmap in Pixels        | 000003E0   |        992 | px Height of Bitmap
Number of color planes being used | 0001       |          1 | Number of color planes being used
bits/pixels          

In [8]:
bmp_header ="42 4D 4E 1B 1D 00 00 00 00 00 56 00 00 00 28 00 00 00 AA 10 00 00 7D 03 00 00 01 00 04 00 00 00 00 00 F8 1A 1D 00 5D 37 00 00 5D 37 00 00 08 00 00 00 00 00 00 00 FF FF FF 00 DB DB DB 00 B7 B7 B7 00 92 92 92 00 6E 6E 6E 00 49 49 49 00 25 25 25 00 00 00 00 00 00 01 00 00 00 00 10 00 01 00 10 00 00 00 00 01 00 00 10 00 00 00 00 00 01 00 00 00 10 00 00 00 00 00 00 00 00 00 00 00 00 00 00 10 00 00 01 00 00 00 00 01 00 00 00 00 00 00 00 00 00 00 00 01 00 01 00 00 00 00 00 00 01 00 01 00 00 10 00 00 00 00 00 00 00 00 00 00 00 00"
bmp_header_analysis(bmp_header)

Field name                        | Hex Value  | Add Value  | Comment
----------------------------------+------------+------------+----------------------------------------
BMP Header Field                  | 4D42       | BM         | BMP Header Field detected => CORRECT
BMP file size                     | 001D1B4E   |     1.8192 | Mbytes Filesize
Unused 1                          | 0000       | -          | Unused Application specific settings
Unused 2                          | 0000       | -          | Unused Application specific settings
Offset of BMP Data                | 00000056   |         86 | Offset of BMP Data
Number of Bytes in DIP Header     | 00000028   |         40 | Number of Bytes in DIP Header
Width of Bitmap in Pixels         | 000010AA   |       4266 | px Width of Bitmap
Height of Bitmap in Pixels        | 0000037D   |        893 | px Height of Bitmap
Number of color planes being used | 0001       |          1 | Number of color planes being used
bits/pixels          

In [10]:
bmp_header = "42 4D 8A 00 DC 00 00 00 00 00 8A 00 00 00 6C 00 00 00 00 14 00 00 00 16 00 00 01 00 04 00 00 00 00 00 00 00 DC 00 46 5C 00 00 46 5C 00 00 04 00 00 00 04 00 00 00 42 47 52 73 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 02 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 FD FD FD 00 99 99"
bmp_header_analysis(bmp_header)

Field name                        | Hex Value  | Add Value  | Comment
----------------------------------+------------+------------+----------------------------------------
BMP Header Field                  | 4D42       | BM         | BMP Header Field detected => CORRECT
BMP file size                     | 00DC008A   |    13.7501 | Mbytes Filesize
Unused 1                          | 0000       | -          | Unused Application specific settings
Unused 2                          | 0000       | -          | Unused Application specific settings
Offset of BMP Data                | 0000008A   |        138 | Offset of BMP Data
Number of Bytes in DIP Header     | 0000006C   |        108 | Number of Bytes in DIP Header
Width of Bitmap in Pixels         | 00001400   |       5120 | px Width of Bitmap
Height of Bitmap in Pixels        | 00001600   |       5632 | px Height of Bitmap
Number of color planes being used | 0001       |          1 | Number of color planes being used
bits/pixels          

In [11]:
bmp_header = "42 4D DA 00 00 00 00 00 00 00 8A 00 00 00 6C 00 00 00 0F 00 00 00 0A 00 00 00 01 00 04 00 00 00 00 00 50 00 00 00 0C 53 00 00 5D 37 00 00 04 00 00 00 04 00 00 00 42 47 52 73 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 02 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 FF FF FF 00 AA AA AA 00 55 55 55 00 00 00 00 00 33 33 33 33 33 00 00 00 33 33 33 33 33 00 00 00 33 33 33 33 33 00 00 00 33 33 33 33 33 00 00 00 33 33 33 33 33 00"
bmp_header_analysis(bmp_header)

Field name                        | Hex Value  | Add Value  | Comment
----------------------------------+------------+------------+----------------------------------------
BMP Header Field                  | 4D42       | BM         | BMP Header Field detected => CORRECT
BMP file size                     | 000000DA   |        218 | Filesize in bytes
Unused 1                          | 0000       | -          | Unused Application specific settings
Unused 2                          | 0000       | -          | Unused Application specific settings
Offset of BMP Data                | 0000008A   |        138 | Offset of BMP Data
Number of Bytes in DIP Header     | 0000006C   |        108 | Number of Bytes in DIP Header
Width of Bitmap in Pixels         | 0000000F   |         15 | px Width of Bitmap
Height of Bitmap in Pixels        | 0000000A   |         10 | px Height of Bitmap
Number of color planes being used | 0001       |          1 | Number of color planes being used
bits/pixels        